In [2]:
import os
import warnings
import sys

In [3]:
import numpy as np
import pandas as pd

In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

/home/shuklanish/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
import mlflow
import mlflow.sklearn

In [6]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4582,6.3,0.37,0.51,6.3,0.048,35.0,146.0,0.99430,3.10,1.01,10.5,6
1082,6.3,0.27,0.25,5.8,0.038,52.0,155.0,0.99500,3.28,0.38,9.4,6
3815,6.3,0.43,0.32,8.8,0.042,18.0,106.0,0.99172,3.28,0.33,12.9,7
3837,8.0,0.27,0.33,1.2,0.050,41.0,103.0,0.99002,3.00,0.45,12.4,6
1820,6.8,0.30,0.35,2.8,0.038,10.0,164.0,0.99120,3.09,0.53,12.0,6
2258,7.6,0.48,0.28,10.4,0.049,57.0,205.0,0.99748,3.24,0.45,9.3,5
1427,7.4,0.25,0.49,1.1,0.042,35.0,156.0,0.99170,3.13,0.55,11.3,5
4472,7.0,0.28,0.28,1.4,0.039,12.0,83.0,0.99173,3.18,0.65,11.1,5
419,7.4,0.31,0.28,1.6,0.050,33.0,137.0,0.99290,3.31,0.56,10.5,6
1160,7.5,0.14,0.34,1.3,0.055,50.0,153.0,0.99450,3.29,0.80,9.6,6


In [13]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(remote_server_uri)

In [14]:
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:5000'

In [15]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

INFO: 'ElasticNet_wine' does not exist. Creating a new experiment


In [16]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [17]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.8222428497595401
  MAE: 0.6278761410160693
  R2: 0.12678721972772666
Save to: mlruns/1/bc8223e4000f4239850b2d0dbcfaab36/artifacts


In [18]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062342
  MAE: 0.6155290394093895
  R2: 0.20224631822892092
Save to: mlruns/1/33dccadacbcf47489cb048565287358c/artifacts


In [19]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: mlruns/1/03c4a47e23c94a799a5b6c80d1ab8ee3/artifacts
